# Analysis Code

In [1]:
import requests
import json

def fetch_news(search_keyword, start_date, end_date, sort_method):
    api_key = "fcf6368111ce48c3b234b0a479d1dca6"

    url = (
        "https://newsapi.org/v2/everything?"
        f"q={search_keyword}&"
        f"from={start_date}&"
        f"to={end_date}&"
        f"sortBy={sort_method}&"
        f"apiKey={api_key}"
    )

    response = requests.get(url)

    if response.status_code == 200:
        fetch_news_data = response.json()

        with open('news_data.json', 'w') as f:
            json.dump(fetch_news_data, f, indent=4)
        print("Data saved to 'news_data.json'")
        return fetch_news_data
    else:
        print(f"Failed to fetch data: {response.status_code} - {response.text}")
        return None

In [2]:
def extract_news_data(title, description, content):
    # Combine title, description, and content
    combined_text = f"{title} {description} {content}".strip()
    return combined_text

In [3]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
from IPython.display import display, Markdown
import re

# LLM model
cls_model = T5ForConditionalGeneration
cls_tokenizer = T5Tokenizer
device_map="auto"
torch_dtype=torch.float16

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")

model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large",
                                        torch_dtype=torch_dtype,
                                        device_map=device_map,
                                        trust_remote_code=True,
                                        # cache_dir = "/work/pi_dhruveshpate_umass_edu/aamballa_umass_edu/models/.cache"
                                        )

tokenizer.pad_token =  tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id
model.eval()


/work/pi_mccallum_umass_edu/gakkinapalli_umass_edu/.conda/envs/semdiv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The argument `trust_remote_code` is to be used with Auto classes. It has no e

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=False)
       

In [4]:
def analyze_news(news_content, ticker):    
    prompt = f"""
    Provide the sentiment of the below {ticker} article as either positive, negative or neutral:

    {news_content}
    """
    # tokenize
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

    generated_ids = model.generate(input_ids = input_ids, max_new_tokens = 100)

    analyze_news_data = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]    
    # display(Markdown(analyze_news_data))

    return analyze_news_data

def clean_sentiment(content):
    # Remove padding tokens
    sentiment = analyze_news(content, 'apple stock')
    return sentiment.replace("<pad>", "").strip()  


In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, MapType, ArrayType, IntegerType 
from pyspark.sql.functions import col, udf

def sentiment_wrapper(articles):
    """
    Process news articles, apply sentiment analysis, and store results in a JSON file.
    """
    if not articles:
        print("No articles found in the data.")
        return

    # Create a SparkSession
    spark = SparkSession.builder \
        .master('local[*]') \
        .config("spark.driver.memory", "15g") \
        .appName('SentimentAnalysis') \
        .getOrCreate()
    
    # Define schema for DataFrame based on article keys
    schema = StructType([
        StructField("source", MapType(StringType(), StringType()), True),
        StructField("author", StringType(), True),
        StructField("title", StringType(), True),
        StructField("description", StringType(), True),
        StructField("url", StringType(), True),
        StructField("urlToImage", StringType(), True),
        StructField("publishedAt", StringType(), True),
        StructField("content", StringType(), True),
    ])
    
    # Create a DataFrame from articles
    df = spark.createDataFrame(articles, schema)

    # Register `extract_news_data` as UDF to combine article fields into a key
    extract_news_udf = udf(extract_news_data, StringType())
    df = df.withColumn("key", extract_news_udf(col("title"), col("description"), col("content")))

    # Define and register UDF for sentiment analysis on combined text 
    sentiment_analysis_udf = udf(clean_sentiment, StringType())
    df = df.withColumn("sentiment", sentiment_analysis_udf(df["key"]))
    
    # Save results to JSON
    output_file_path = "sentiment_data.json"
    df_list = df.select("sentiment").rdd.map(lambda row: row["sentiment"]).collect()
    
    with open(output_file_path, "w") as json_file:
        json.dump(df_list, json_file, indent=2)

    # Stop SparkSession
    spark.stop()

    return output_file_path


In [6]:
def main():
    fetch_news_data = fetch_news('apple stock', '2024-10-16', '2024-10-17', 'popularity')

    if fetch_news_data and "articles" in fetch_news_data:
        articles = fetch_news_data["articles"]
        sentiment_output_path = sentiment_wrapper(articles)
        print(f"Sentiment analysis results saved to {sentiment_output_path}")
    else:
        print("No articles found or failed to fetch news data.")


In [7]:
if __name__ == "__main__":
    main()

Data saved to 'news_data.json'


24/11/16 03:36:12 WARN Utils: Your hostname, gpu013 resolves to a loopback address: 127.0.1.1; using 10.100.20.13 instead (on interface en0)
24/11/16 03:36:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/16 03:36:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Sentiment analysis results saved to sentiment_data.json


# Memory Usage and Throughput

### Execute the memory usage code for the first time

In [6]:
import psutil
import GPUtil
import time

def monitor_memory_usage(func, *args, **kwargs):
    """
    Monitors memory usage of CPU and GPU during the execution of the given function.

    Args:
        func (function): The main function to execute.
        *args: Positional arguments to pass to the function.
        **kwargs: Keyword arguments to pass to the function.

    Returns:
        tuple: The output of the executed function and the execution time in seconds.
    """
    start_time = time.time()

    result = func(*args, **kwargs)

    # Get total and used memory (RAM)
    memory_info = psutil.virtual_memory()
    memory_used = (memory_info.total - memory_info.available) / (1024 ** 3)  # Convert to GB

    # Print CPU usage in GB (use virtual memory usage for CPU context)
    cpu_memory_used = memory_used

    gpus = GPUtil.getGPUs()
    if gpus:
        gpu_memory_used = gpus[0].memoryUsed / 1024  # Convert MB to GB
    else:
        gpu_memory_used = 0

    print(f"CPU Memory Usage: {cpu_memory_used:.2f} GB")
    print(f"GPU Memory Usage: {gpu_memory_used:.2f} GB")

    end_time = time.time()

    print(f"Response Time: {end_time - start_time:.2f} seconds")

    return result, end_time - start_time  # Return result and execution time


def calculate_throughput(response_time, num_articles):
    """
    Calculates throughput based on the formula: response_time / number_of_articles.

    Args:
        response_time (float): The total time taken for processing in seconds.
        num_articles (int): The number of articles processed.

    Returns:
        float: The throughput (in seconds per article).
    """
    if num_articles == 0:
        return 0 
    return response_time / num_articles


def main():

    fetch_news_data = fetch_news('apple stock', '2024-10-16', '2024-10-17', 'popularity')
    
    if fetch_news_data and "articles" in fetch_news_data:
        articles = fetch_news_data["articles"]
        output, response_time = monitor_memory_usage(sentiment_wrapper, articles)
        # print(f"Sentiment analysis results saved to {output}")
    else:
        print("No articles found or failed to fetch news data.")
    articles = fetch_news_data["articles"]

    throughput = calculate_throughput(response_time, len(articles))
    print(f"Throughput (seconds per article): {throughput:.4f}")


# Run the main function
if __name__ == "__main__":
    main()


Data saved to 'news_data.json'


24/11/16 03:42:11 WARN Utils: Your hostname, gpu013 resolves to a loopback address: 127.0.1.1; using 10.100.20.13 instead (on interface en0)
24/11/16 03:42:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/16 03:42:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


CPU Memory Usage: 227.12 GB
GPU Memory Usage: 2.23 GB
Response Time: 33.64 seconds
Throughput (seconds per article): 0.3364


### Execute the memory usage code using cache

In [8]:
import psutil
import GPUtil
import time

def monitor_memory_usage(func, *args, **kwargs):
    """
    Monitors memory usage of CPU and GPU during the execution of the given function.

    Args:
        func (function): The main function to execute.
        *args: Positional arguments to pass to the function.
        **kwargs: Keyword arguments to pass to the function.

    Returns:
        tuple: The output of the executed function and the execution time in seconds.
    """
    start_time = time.time()

    result = func(*args, **kwargs)

    # Get total and used memory (RAM)
    memory_info = psutil.virtual_memory()
    memory_used = (memory_info.total - memory_info.available) / (1024 ** 3)  # Convert to GB

    # Print CPU usage in GB (use virtual memory usage for CPU context)
    cpu_memory_used = memory_used

    gpus = GPUtil.getGPUs()
    if gpus:
        gpu_memory_used = gpus[0].memoryUsed / 1024  # Convert MB to GB
    else:
        gpu_memory_used = 0

    print(f"CPU Memory Usage: {cpu_memory_used:.2f} GB")
    print(f"GPU Memory Usage: {gpu_memory_used:.2f} GB")

    end_time = time.time()

    print(f"Response Time: {end_time - start_time:.2f} seconds")

    return result, end_time - start_time  # Return result and execution time


def calculate_throughput(response_time, num_articles):
    """
    Calculates throughput based on the formula: response_time / number_of_articles.

    Args:
        response_time (float): The total time taken for processing in seconds.
        num_articles (int): The number of articles processed.

    Returns:
        float: The throughput (in seconds per article).
    """
    if num_articles == 0:
        return 0 
    return response_time / num_articles


def main():

    fetch_news_data = fetch_news('apple stock', '2024-10-16', '2024-10-17', 'popularity')
    
    if fetch_news_data and "articles" in fetch_news_data:
        articles = fetch_news_data["articles"]
        output, response_time = monitor_memory_usage(sentiment_wrapper, articles)
        # print(f"Sentiment analysis results saved to {output}")
    else:
        print("No articles found or failed to fetch news data.")
    articles = fetch_news_data["articles"]

    throughput = calculate_throughput(response_time, len(articles))
    print(f"Throughput (seconds per article): {throughput:.4f}")


# Run the main function
if __name__ == "__main__":
    main()


Data saved to 'news_data.json'


CPU Memory Usage: 222.37 GB
GPU Memory Usage: 2.23 GB
Response Time: 22.84 seconds
Throughput (seconds per article): 0.2284


# Accuracy

In [7]:
import openai
from openai import OpenAI
import os

def analyze_news_openai(news_content, api_key, ticker):
    openai.api_key = api_key
    results = []
    for news in news_content:
        instruction = f"Provide the sentiment of the below {ticker} article as either positive, negative or neutral:"
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "user",
                    "content": f"{news}\n{instruction}\n"
                }
            ]
        )
        # Get the response content
        sentiment_response = completion.choices[0].message.content.strip()

        # Extract the sentiment (positive, negative, neutral)
        match = re.search(r"(positive|negative|neutral)", sentiment_response, re.IGNORECASE)
        sentiment_extract = match.group() if match else "unknown"  # Default to "unknown" if no match found
        results.append(sentiment_extract)

    # Save the results to JSON
    with open('analyze_news_openai.json', 'w') as f:
        json.dump(results, f, indent=4)
    print("Data saved to 'analyze_news_openai.json'")

    return results

In [ ]:
if __name__ == "__main__":
    # use the api key for openai, not suggested to push the code with key
    
    client = OpenAI()

    file_path = "news_data.json"
    with open(file_path, 'r') as file:
        fetch_news_data = json.load(file)

    news_data = []
    for article in fetch_news_data["articles"]:
        extracted_news_data = extract_news_data(article["title"], article["description"], article["content"])
        extract_news_data=extract_news_api(fetch_news_data)
        news_data.append(extract_news_data)
        
    sentiments = analyze_news_openai(news_data, openai_api_key)
    print(sentiments)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
from tabulate import tabulate

predicted_label_file_path = "sentiment_data.json"
with open(predicted_label_file_path, 'r', encoding='utf-8') as predicted_label:
    predicted_labels = json.load(predicted_label)

true_label_file_path = "analyze_news_openai.json"
with open(true_label_file_path, 'r', encoding='utf-8') as true_label:
    true_labels = json.load(true_label)

conf_matrix = confusion_matrix(true_labels, predicted_labels, labels=["positive", "negative", "neutral"])

print("Confusion Matrix:")
print(tabulate(conf_matrix, headers=["Predicted Positive", "Predicted Negative", "Predicted Neutral"],
               tablefmt="pretty", showindex=["Actual Positive", "Actual Negative", "Actual Neutral"]))

accuracy = accuracy_score(true_labels, predicted_labels)
print(f"\nAccuracy: {accuracy * 100:.2f}%")